In [10]:
#titanic_kfp_pipeline.ipynb
#Copyright 2020 Google LLC. 
#This software is provided as-is, without warranty or representation for any use or purpose. 
#Your use of it is subject to your agreements with Google.
#Author: whjang@google.com

PROJECT_ID = "learned-iridium-385115"
IMAGE_PREFIX = "whjang-titanic"
PREPROC_DIR = "preprocess"
TRAIN_DIR = "train"

WORK_BUCKET = "gs://practice-tensorflow/3.tutorial-kubeflow"
RAW_CSV_GCS_URI = WORK_BUCKET + "/rawdata/train.csv"
PREPROC_CSV_GCS_URI = WORK_BUCKET + "/preprocdata/processed_train.csv"
ACC_CSV_GCS_URI = WORK_BUCKET + "/latestacc/accuracy.csv"
MODEL_PKL_GCS_URI = WORK_BUCKET + "/model/model.pkl"
STAGE_GCS_FOLDER = WORK_BUCKET + "/stage"

AIPJOB_TRAINER_GCS_PATH = WORK_BUCKET + "/train/titanic_train.tar.gz"
AIPJOB_OUTPUT_GCS_PATH = WORK_BUCKET + "/train/output/"

import os
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["IMAGE_PREFIX"] = IMAGE_PREFIX
os.environ["PREPROC_DIR"] = PREPROC_DIR
os.environ["TRAIN_DIR"] = TRAIN_DIR

os.environ["WORK_BUCKET"] = WORK_BUCKET
os.environ["RAW_CSV_GCS_URI"] = RAW_CSV_GCS_URI
os.environ["PREPROC_CSV_GCS_URI"] = PREPROC_CSV_GCS_URI
os.environ["ACC_CSV_GCS_URI"] = ACC_CSV_GCS_URI
os.environ["MODEL_PKL_GCS_URI"] = MODEL_PKL_GCS_URI
os.environ["STAGE_GCS_FOLDER"] = STAGE_GCS_FOLDER
os.environ["AIPJOB_TRAINER_GCS_PATH"] = AIPJOB_TRAINER_GCS_PATH
os.environ["AIPJOB_OUTPUT_GCS_PATH"] = AIPJOB_OUTPUT_GCS_PATH

!echo $PROJECT_ID
!echo $IMAGE_PREFIX
!echo $PREPROC_DIR
!echo $TRAIN_DIR
!echo $WORK_BUCKET
!echo $RAW_CSV_GCS_URI
!echo $PREPROC_CSV_GCS_URI
!echo $ACC_CSV_GCS_URI
!echo $MODEL_PKL_GCS_URI
!echo $STAGE_GCS_FOLDER
!echo $AIPJOB_TRAINER_GCS_PATH
!echo $AIPJOB_OUTPUT_GCS_PATH

learned-iridium-385115
whjang-titanic
preprocess
train
gs://practice-tensorflow/3.tutorial-kubeflow
gs://practice-tensorflow/3.tutorial-kubeflow/rawdata/train.csv
gs://practice-tensorflow/3.tutorial-kubeflow/preprocdata/processed_train.csv
gs://practice-tensorflow/3.tutorial-kubeflow/latestacc/accuracy.csv
gs://practice-tensorflow/3.tutorial-kubeflow/model/model.pkl
gs://practice-tensorflow/3.tutorial-kubeflow/stage
gs://practice-tensorflow/3.tutorial-kubeflow/train/titanic_train.tar.gz
gs://practice-tensorflow/3.tutorial-kubeflow/train/output/


In [12]:
!docker build -t $IMAGE_PREFIX-$PREPROC_DIR $PREPROC_DIR/.
!docker tag $IMAGE_PREFIX-$PREPROC_DIR:latest gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$PREPROC_DIR:latest
!docker push gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$PREPROC_DIR:latest

Sending build context to Docker daemon   7.68kB
Step 1/10 : FROM amancevice/pandas
 ---> 913743377700
Step 2/10 : RUN pip3 install --upgrade pandas
 ---> Using cache
 ---> c50a88bd7679
Step 3/10 : RUN pip3 install --upgrade gcsfs
 ---> Using cache
 ---> e3937fb8d6ce
Step 4/10 : RUN pip3 install --upgrade argparse
 ---> Using cache
 ---> ca55437f722b
Step 5/10 : RUN pip3 install sklearn
 ---> Using cache
 ---> e1bca2a336e5
Step 6/10 : ENV PYTHONUNBUFFERED 1
 ---> Using cache
 ---> 839afde6bcb5
Step 7/10 : RUN mkdir -p /titanic/src
 ---> Using cache
 ---> cd2b35fb86b5
Step 8/10 : COPY . /titanic/src
 ---> Using cache
 ---> 15dfff23d5c0
Step 9/10 : WORKDIR /titanic/src
 ---> Using cache
 ---> f4ab1b99ef4d
Step 10/10 : ENTRYPOINT ["python", "titanic_preprocess.py"]
 ---> Using cache
 ---> d364eb9577a0
Successfully built d364eb9577a0
Successfully tagged whjang-titanic-preprocess:latest
The push refers to repository [gcr.io/learned-iridium-385115/whjang-titanic-preprocess]

f8663698: Prepari

In [7]:
!docker image ls

REPOSITORY                               TAG       IMAGE ID       CREATED         SIZE
whjang-titanic-preprocess                latest    d364eb9577a0   2 minutes ago   1.14GB
amancevice/pandas                        latest    913743377700   9 days ago      1.11GB
frolvlad/alpine-python-machinelearning   latest    edb47bd01cc6   2 months ago    399MB


In [13]:
!docker run gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$PREPROC_DIR:latest --raw_csv_gcs_uri $RAW_CSV_GCS_URI --preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI

Preprocessing Titanic Data
   Pclass  SibSp  Sex_female  Sex_male  Survived
0       3      1       False      True         0
1       1      1        True     False         1
2       3      0        True     False         1
3       1      1        True     False         1
4       3      0       False      True         0


# 모델 train, evaluation

In [15]:
!echo $PREPROC_CSV_GCS_URI
!echo $MODEL_PKL_GCS_URI
!echo $ACC_CSV_GCS_URI

gs://practice-tensorflow/3.tutorial-kubeflow/preprocdata/processed_train.csv
gs://practice-tensorflow/3.tutorial-kubeflow/model/model.pkl
gs://practice-tensorflow/3.tutorial-kubeflow/latestacc/accuracy.csv


In [19]:
!docker build -t $IMAGE_PREFIX-$TRAIN_DIR $TRAIN_DIR/.
!docker tag $IMAGE_PREFIX-$TRAIN_DIR:latest gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker push gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker run gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest \
--preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI \
--model_pkl_gcs_uri $MODEL_PKL_GCS_URI \
--acc_csv_gcs_uri $ACC_CSV_GCS_URI \
--min_acc_progress 0.000001

Sending build context to Docker daemon  14.85kB
Step 1/12 : FROM amancevice/pandas
 ---> 913743377700
Step 2/12 : RUN pip3 install --upgrade google-cloud-storage
 ---> Using cache
 ---> 879ce7aae86c
Step 3/12 : RUN pip3 install --upgrade gcsfs
 ---> Using cache
 ---> d7104851174f
Step 4/12 : RUN pip3 install --upgrade scikit-learn
 ---> Using cache
 ---> 43744646c9db
Step 5/12 : RUN pip3 install --upgrade argparse
 ---> Using cache
 ---> cdbbfbc9b3fe
Step 6/12 : RUN pip3 install --upgrade pandas
 ---> Using cache
 ---> a1a30c2227ca
Step 7/12 : RUN pip3 install --upgrade simplejson
 ---> Using cache
 ---> 2b6d9307faa5
Step 8/12 : ENV PYTHONUNBUFFERED 1
 ---> Using cache
 ---> 46c41aab76d1
Step 9/12 : RUN mkdir -p /titanic/src
 ---> Using cache
 ---> 21d3e2614cc7
Step 10/12 : COPY . /titanic/src
 ---> 71d9c3dff8fa
Step 11/12 : WORKDIR /titanic/src
 ---> Running in 7ff7cfafb1b9
Removing intermediate container 7ff7cfafb1b9
 ---> b24bfc41e757
Step 12/12 : ENTRYPOINT ["python", "titanic_trai

# pipeline setup

In [ ]:
!rm -fr titanic_train.tar.gz
!tar zcvf titanic_train.tar.gz *
!gsutil cp titanic_train.tar.gz $AIPJOB_TRAINER_GCS_PATH

# Kubeflow Pipeline 구성 코드 작성, AI Platform Pipeline 등록 및 실행

In [2]:
#titanic_kfp_pipeline.ipynb
#Copyright 2020 Google LLC.
#This software is provided as-is, without warranty or representation for any use or purpose.
#Your use of it is subject to your agreements with Google.
#Author: whjang@google.com
#!pip3 install -U kfp
import kfp
import kfp.components as comp
from kfp import dsl
from kfp import compiler
from kfp.components import func_to_container_op
import time
import datetime
import json

PIPELINE_HOST = "7f292c1c27803874-dot-us-central1.pipelines.googleusercontent.com"
WORK_BUCKET = "gs://practice-tensorflow"
EXPERIMENT_NAME = "Titanic Draft Experiment"

# Function for determine deployment
@func_to_container_op
def check_and_deploy_op(ACC_CSV_GCS_URI) -> str:
    import sys, subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'gcsfs'])
    import pandas as pd
    acc_df = pd.read_csv(ACC_CSV_GCS_URI)
    return acc_df["deploy"].item()

@func_to_container_op
def finish_deploy_op(ACC_CSV_GCS_URI):
    import sys, subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'gcsfs'])
    import pandas as pd
    acc_df = pd.read_csv(ACC_CSV_GCS_URI)
    acc_df["deploy"] = "done"
    acc_df.to_csv(ACC_CSV_GCS_URI)
    print("Successfully new model was deployed")

@dsl.pipeline(
    name="titanic-kubeflow-pipeline-demo",
    description="Titanic Kubeflow Pipeline demo embrassing AI Platform in Google Cloud"
)
def titanic_pipeline(
        PROJECT_ID,
        WORK_BUCKET,
        RAW_CSV_GCS_URI,
        PREPROC_CSV_GCS_URI,
        ACC_CSV_GCS_URI,
        MODEL_PKL_GCS_URI,
        MIN_ACC_PROGRESS,
        STAGE_GCS_FOLDER,
        TRAIN_ON_CLOUD,
        AIPJOB_TRAINER_GCS_PATH,
        AIPJOB_OUTPUT_GCS_PATH
):
    IMAGE_PREFIX = "whjang-titanic"
    PREPROC_DIR = "preprocess"
    TRAIN_DIR = "train"
    MODEL_DIR = "model"

    preprocess = dsl.ContainerOp(
        name = "Preprocess raw data and generate new one",
        image = f"gcr.io/{str(PROJECT_ID)}/{IMAGE_PREFIX}-{PREPROC_DIR}:latest",
        arguments=[
            "--raw_csv_gcs_uri", RAW_CSV_GCS_URI,
            "--preproc_csv_gcs_uri", PREPROC_CSV_GCS_URI
        ]
    )

    train_args = [
        "--preproc_csv_gcs_uri", str(PREPROC_CSV_GCS_URI),
        "--model_pkl_gcs_uri", str(MODEL_PKL_GCS_URI),
        "--acc_csv_gcs_uri", str(ACC_CSV_GCS_URI),
        "--min_acc_progress", str(MIN_ACC_PROGRESS)
    ]

    with dsl.Condition(TRAIN_ON_CLOUD == False) as check_condition1:
        train = dsl.ContainerOp(
            name="Train",
            image = f"gcr.io/{str(PROJECT_ID)}/{IMAGE_PREFIX}-{TRAIN_DIR}:latest",
            arguments = train_args,
            file_outputs={
                "mlpipeline-metrics": "/mlpipeline-metrics.json"
            }
        )

    with dsl.Condition(TRAIN_ON_CLOUD == True) as check_condition2:
        aip_job_train_op = comp.load_component_from_url("https://raw.githubusercontent.com/kubeflow/pipelines/1.0.0/components/gcp/ml_engine/train/component.yaml")
        train = dsl.ContainerOp(
            name="Train",
            image = f"gcr.io/{str(PROJECT_ID)}/{IMAGE_PREFIX}-{TRAIN_DIR}:latest",
            arguments = train_args,
            file_outputs={
                "mlpipeline-metrics": "/mlpipeline-metrics.json"
            }
        )
        help(aip_job_train_op)
        aip_train = aip_job_train_op(
            project_id=PROJECT_ID,
            python_module="train.titanic_train",
            package_uris=json.dumps([str(AIPJOB_TRAINER_GCS_PATH)]),
            region="us-central1",
            args=json.dumps(train_args),
            job_dir=AIPJOB_OUTPUT_GCS_PATH,
            python_version="3.7",
            runtime_version="1.15", #cf. 2.1
            master_image_uri="",
            worker_image_uri="",
            training_input="",
            job_id_prefix="",
            job_id="",
            wait_interval=5
        )

    check_deploy = check_and_deploy_op(ACC_CSV_GCS_URI)
    with dsl.Condition(check_deploy.ouput == "pending"):
        aip_model_deploy_op = comp.load_component_from_url("https://raw.githubusercontent.com/kubeflow/pipelines/1.0.0/components/gcp/ml_engine/deploy/component.yaml")
        help(aip_model_deploy_op)
        aip_model_deploy = aip_model_deploy_op(
            model_uri=str(WORK_BUCKET) + "/" + MODEL_DIR,
            project_id=PROJECT_ID,
            model_id="",
            version_id="",
            runtime_version="1.15",  #cf. 2.1
            python_version="3.7",
            version="",
            replace_existing_version="False",
            set_default="True",
            wait_interval=5
        )
    lastStep = finish_deploy_op(ACC_CSV_GCS_URI)

    check_condition1.after(preprocess)
    check_condition2.after(preprocess)
    check_deploy.after(aip_train)
    lastStep.after(aip_model_deploy)

    train.execution_options.caching_strategy.max_cache_staleness = "POD"
    aip_train.execution_options.caching_strategy.max_cache_staleness = "POD"
    check_deploy.execution_options.caching_strategy.max_cache_staleness = "POD"
    aip_model_deploy.execution_options.caching_strategy.max_cache_staleness = "POD"
    lastStep.execution_options.caching_strategy.max_cache_staleness = "POD"

ModuleNotFoundError: No module named 'requests_toolbelt.adapters.appengine'

In [ ]:
args = {
    "PROJECT_ID": "learned-iridium-385115",
    "WORK_BUCKET": WORK_BUCKET,
    "RAW_CSV_GCS_URI": WORK_BUCKET + "/rawdata/train.csv",
    "PREPROC_CSV_GCS_URI": WORK_BUCKET + "/preprocdata/processed_train.csv",
    "ACC_CSV_GCS_URI": WORK_BUCKET + "/latestacc/accuracy.csv",
    "MODEL_PKL_GCS_URI": WORK_BUCKET + "/model/model.pkl",
    "MIN_ACC_PROGRESS": 0.000001,
    "STAGE_GCS_FOLDER": WORK_BUCKET + "/stage",
    "TRAIN_ON_CLOUD": False,
    "AIPJOB_TRAINER_GCS_PATH": WORK_BUCKET + "/train/titanic_train.tar.gz",
    "AIPJOB_OUTPUT_GCS_PATH": WORK_BUCKET + "/train/output/"
}

# Notebook에서 테스트 한 예제이기 때문에, 쉘에서 실행시키시려면 main 함수 형태로 변경하셔야 합니다.
# Dockerize한 전처리 모듈 및 ML 학습, 검증 모듈과 달리 @func_to_container_op를 사용한 lightweights 방식의 함수들도 있습니다. check_and_deploy_op와 finish_deploy_op입니다. @func_to_container_op는 이렇게 함수를 자동으로 Dockerize 하기 때문에 별도의 Dockerfile등을 정의할 필요가 없는 장점이 있지만, 필요한 외부 패키지등은 함수 내부에서 subprocess를 통해 각각 설치해야 합니다.

client = kfp.Client(host=PIPELINE_HOST)
#pipeline_name = "titanic_pipelines.zip"
#compiler.Compiler().compile(titanic_pipeline, pipeline_name)
#try:
# pipeline = client.upload_pipeline(pipeline_package_path=pipeline_name, pipeline_name=pipeline_name)
# print("uploaded:" + pipeline.id)
#except:
# print("already exist")
client.create_run_from_pipeline_func(
    titanic_pipeline,
    arguments=args,
    experiment_name=EXPERIMENT_NAME
)